In [2]:
import requests
import json
import mysql.connector
from database import config
from datetime import date, timedelta, datetime
import binascii
import re
import hanja
import numpy as np
import torch
db = mysql.connector.connect(**config)
cursor = db.cursor()


In [3]:
cursor.execute("select max(createtime) from news_gpt.news_recent;")
print(cursor.fetchall())

[(datetime.datetime(2023, 11, 9, 23, 57),)]


In [2]:
class NoDataException(Exception):
    pass
class ZeroDataException(Exception):
    pass

In [14]:
today0 = date.today()

articles = []
api_dic = {}
for n in range(75,60,-1):  # 대충 2달치. 어제까지
    day0 = today0 - timedelta(days=n)  # 어제(n=1), 오늘(n=0)
    print(f"========={day0}===========")
    url = f'https://openapi.donga.com/newsList?p={day0.strftime("%Y%m%d")}'
    temp = requests.get(url)
    temp = json.loads(temp.content)
    if 'data' not in [*temp]:
        # 그냥 가끔 data 없이 올 떄가 있음. 그리고 새벽에 첫 기사 나오기 전까지는 'data'가 없음.
        print('### NoDataException ###')
        raise NoDataException()
    articles += temp['data']

if len(articles) ==0:
    print('### ZeroDataException ###')
    raise ZeroDataException


=========2023-08-27===========
=========2023-08-28===========
=========2023-08-29===========
=========2023-08-30===========
=========2023-08-31===========
=========2023-09-01===========
=========2023-09-02===========
=========2023-09-03===========
=========2023-09-04===========
=========2023-09-05===========
=========2023-09-06===========
=========2023-09-07===========
=========2023-09-08===========
=========2023-09-09===========
=========2023-09-10===========


In [6]:
print(articles[0])

{'gid': '121624612', 'createtime': '2023-10-11 23:56:00', 'title': '[횡설수설/송평인]대형 로펌 사건 회피로 대법원 ‘민폐’가 된 권영준 대법관', 'content': '대법관 한 명이 한 해 주심을 맡아 처리하는 사건이 지난해 평균 4038건이었다. 2000년대 후반 2000건대에 진입했고 2010년대는 대체로 3000건대였으나 지난해 4000건대로 올라섰다. 매주 77건의 사건을 처리하는 셈이다. ‘월화수목금금금’으로 일해도 매일 11건씩 처리해야 할 정도로 사건이 많기 때문에 상고법원 얘기도 나오고 대법관 증원 얘기도 나온다. ▷권영준 대법관은 7월 중순 취임 후 사건 59건의 주심을 회피했다. 회피는 재판관 본인이 이해관계에 얽혀 있다고 여기는 사건의 취급을 피하는 제도다. 권 대법관은 서울대 법학전문대학원 교수 때 대형 법무법인(로펌) 7곳이 맡은 사건 38건에 대해 법률의견서를 써주고 18억 원을 받았다. 권 대법관은 이 사실이 논란이 되자 국회 인사청문회에서 “대법관이 되면 제가 관여하지 않은 사건이라고 해도 관계를 맺은 로펌의 사건은 2년간 모두 회피 신청하겠다”고 약속했다. 2년으로 한정한 것은 이해충돌방지법에서 이해당사자로 간주되는 기간이 2년이기 때문이다. ▷59건은 대법관 한 명이 두 달간 처리하는 사건의 약 10%에 해당한다. 권 대법관이 약속을 지키려면 매년 350여 건씩, 2년간 700여 건을 회피해야 한다. 물론 본인이 회피하는 사건의 수만큼 다른 사건을 넘겨받기 때문에 처리하는 사건 수는 차이가 없다. 다만 대형 로펌이 수임한 사건은 복잡할 가능성이 커 다른 대법관들이 처리에 더 많은 시간과 노력이 들 수 있다. ▷회피는 본래 사건의 주심을 맡지 않는 것일 뿐만 아니라 선고를 위한 합의에도 참여하지 않는 것이다. 회피를 제대로 한다면 권 대법관은 다른 대법관이 주심을 맡은 사건이 대형 로펌과 관계돼 있을 때도 합의에 참여할 수 없다. 대법원 사건은 일단 소부(小部) 처리가 원칙

In [20]:
db = mysql.connector.connect(**config)
cursor = db.cursor()

# 작업을 수행하기 전 상태 체크
cursor.execute("""
select gid from news_gpt.news_recent
""")
already = np.array(cursor.fetchall()).reshape(1,-1)[0]
# already = list(map(lambda x:x[:-2], already)) # '_0' 떼기
# already = list(set(already)) #중복 제거

In [23]:
already = list(map(lambda x:x[:-2], already)) # '_0' 떼기
already = list(set(already)) #중복 제거

In [24]:
already

['103742541',
 '109501965',
 '94658429',
 '106770255',
 '104678115',
 '96971256',
 '101541911',
 '119716293',
 '116739272',
 '95506302',
 '114984182',
 '93843711',
 '109927197',
 '99063630',
 '107605854',
 '112041337',
 '103255186',
 '120854413',
 '109654711',
 '101997275',
 '108588666',
 '110686668',
 '110144107',
 '101124430',
 '118646546',
 '98956433',
 '109502842',
 '99048125',
 '105688376',
 '97023102',
 '93907554',
 '101401593',
 '109354575',
 '109566717',
 '111279159',
 '119254384',
 '115708300',
 '115743068',
 '101982512',
 '104708647',
 '107294220',
 '94273449',
 '97470943',
 '94011727',
 '97773341',
 '116030175',
 '105007727',
 '105763918',
 '109382922',
 '114268805',
 '94389373',
 '113462460',
 '118130033',
 '106181458',
 '100027287',
 '110733076',
 '101282093',
 '120078967',
 '98438787',
 '118370837',
 '110992832',
 '103819219',
 '94389176',
 '104172688',
 '117457738',
 '101814681',
 '116876224',
 '97679851',
 '99487853',
 '106341545',
 '118812041',
 '93549142',
 '101091670

In [33]:
from tqdm import tqdm
for ar in articles:
    if ar['source'] in ['동아일보', '동아닷컴', '신동아', '주간동아', '여성동아', '스포츠동아'] and len(ar['content']) > 500 :
        gid_ori = ar['gid']
        print(gid_ori not in already , end=' ')
        # if gid_ori not in already:
        
            # print(ar['createtime'])
            # print(ar['gid'])
            # print(ar['title'])
            # print(ar['source'])
            # print(ar['content'])
            # break

False False False False False False True False False False False False False False False False False False False False False False False False True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False True False False False False False True False False False False False False False False False False False False False False False False False False False False False False False False False False True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False Fal

KeyboardInterrupt: 

In [12]:
cursor.execute("""
select max(createtime) from news_gpt.news_recent
""")
print(cursor.fetchall())

[(datetime.datetime(2023, 9, 4, 23, 57),)]


In [27]:
'103742541' in already

True